# Getting Started with Microsoft's Presidio: A Step-by-Step Guide to Detecting and Anonymizing PII in Text
In this tutorial, we will explore how to use Microsoft's Presidio, an open-source framework for detecting, analyzing, and anonymizing personally identifiable information (PII) in free-form text. You’ll learn how to:

* Set up and install the necessary Presidio packages

* Detect common PII entities such as names, phone numbers, and credit card details

* Define custom recognizers for domain-specific entities (e.g., PAN, Aadhaar)

* Create and register custom anonymizers (like hashing or pseudonymization)

* Reuse anonymization mappings for consistent re-anonymization




## 1. Installing the libraries

To get started with Presidio, you'll need to install the following key libraries:

* **presidio-analyzer**: This is the core library responsible for detecting PII entities in text using built-in and custom recognizers.

* **presidio-anonymizer:** This library provides tools to anonymize (e.g., redact, replace, hash) the detected PII using configurable operators.

* **spaCy NLP model (en_core_web_lg):** Presidio uses spaCy under the hood for natural language processing tasks like named entity recognition. The en_core_web_lg model provides high-accuracy results and is recommended for English-language PII detection.

In [2]:
!pip install presidio-analyzer
!pip install presidio-anonymizer
!python -m spacy download en_core_web_lg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 2.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


You might need to restart the session to install the libraries.

# 2. Presidio Analyzer

## 2.1 Basic PII Detection
In this block, we initialize the Presidio Analyzer Engine and run a basic analysis to detect a U.S. phone number from a sample text. We also suppress lower-level log warnings from the Presidio library for cleaner output.

The AnalyzerEngine loads spaCy’s NLP pipeline and predefined recognizers to scan the input text for sensitive entities. In this example, we specify PHONE_NUMBER as the target entity.

In [2]:
import logging
logging.getLogger("presidio-analyzer").setLevel(logging.ERROR)

from presidio_analyzer import AnalyzerEngine

# Set up the engine, loads the NLP module (spaCy model by default) and other PII recognizers
analyzer = AnalyzerEngine()

# Call analyzer to get results
results = analyzer.analyze(text="My phone number is 212-555-5555",
                           entities=["PHONE_NUMBER"],
                           language='en')
print(results)

[type: PHONE_NUMBER, start: 19, end: 31, score: 0.75]


## 2.2 Creating a Custom PII Recognizer with a Deny List (Academic Titles)
This code block shows how to create a custom PII recognizer in Presidio using a simple deny list, ideal for detecting fixed terms like academic titles (e.g., "Dr.", "Prof."). The recognizer is added to Presidio’s registry and used by the analyzer to scan input text.

While this tutorial covers only the deny list approach, Presidio also supports regex-based patterns, NLP models, and external recognizers. For those advanced methods, refer to the official docs: [Adding Custom Recognizers.](https://microsoft.github.io/presidio/analyzer/adding_recognizers/)

In [3]:
from presidio_analyzer import AnalyzerEngine, PatternRecognizer, RecognizerRegistry

# Step 1: Create a custom pattern recognizer using deny_list
academic_title_recognizer = PatternRecognizer(
    supported_entity="ACADEMIC_TITLE",
    deny_list=["Dr.", "Dr", "Professor", "Prof."]
)

# Step 2: Add it to a registry
registry = RecognizerRegistry()
registry.load_predefined_recognizers()
registry.add_recognizer(academic_title_recognizer)

# Step 3: Create analyzer engine with the updated registry
analyzer = AnalyzerEngine(registry=registry)

# Step 4: Analyze text
text = "Prof. John Smith is meeting with Dr. Alice Brown."
results = analyzer.analyze(text=text, language="en")

for result in results:
    print(result)

type: ACADEMIC_TITLE, start: 0, end: 5, score: 1.0
type: ACADEMIC_TITLE, start: 33, end: 36, score: 1.0
type: PERSON, start: 6, end: 16, score: 0.85
type: PERSON, start: 37, end: 48, score: 0.85


# 3. Presidio Anonymizer

## 3.1 Anonymizing PII Using Presidio Anonymizer
This code block demonstrates how to use the Presidio Anonymizer Engine to anonymize detected PII entities in a given text. In this example, we manually define two PERSON entities using RecognizerResult, simulating output from the Presidio Analyzer. These entities represent the names "Bond" and "James Bond" in the sample text.

We use the "replace" operator to substitute both names with a placeholder value ("BIP"), effectively anonymizing the sensitive data. This is done by passing an OperatorConfig with the desired anonymization strategy (replace) to the AnonymizerEngine.

This pattern can be easily extended to apply other built-in operations like "redact", "hash", or custom pseudonymization strategies.

In [10]:
from presidio_anonymizer import AnonymizerEngine
from presidio_anonymizer.entities import RecognizerResult, OperatorConfig

# Initialize the engine:
engine = AnonymizerEngine()

# Invoke the anonymize function with the text,
# analyzer results (potentially coming from presidio-analyzer) and
# Operators to get the anonymization output:
result = engine.anonymize(
    text="My name is Bond, James Bond",
    analyzer_results=[
        RecognizerResult(entity_type="PERSON", start=11, end=15, score=0.8),
        RecognizerResult(entity_type="PERSON", start=17, end=27, score=0.8),
    ],
    operators={"PERSON": OperatorConfig("replace", {"new_value": "BIP"})},
)

print(result)

text: My name is BIP, BIP
items:
[
    {'start': 16, 'end': 19, 'entity_type': 'PERSON', 'text': 'BIP', 'operator': 'replace'},
    {'start': 11, 'end': 14, 'entity_type': 'PERSON', 'text': 'BIP', 'operator': 'replace'}
]



# 4. Custom Entity Recognition, Hash-Based Anonymization, and Consistent Re-Anonymization with Presidio
In this example, we take Presidio a step further by demonstrating:

* ✅ Defining custom PII entities (e.g., Aadhaar and PAN numbers) using regex-based PatternRecognizers

* 🔐 Anonymizing sensitive data using a custom hash-based operator (ReAnonymizer)

* ♻️ Re-anonymizing the same values consistently across multiple texts by maintaining a mapping of original → hashed values

We implement a custom ReAnonymizer operator that checks if a given value has already been hashed and reuses the same output to preserve consistency. This is particularly useful when anonymized data needs to retain some utility — for example, linking records by pseudonymous IDs.

## 4.1 Define a Custom Hash-Based Anonymizer (ReAnonymizer)
This block defines a custom Operator called ReAnonymizer that uses SHA-256 hashing to anonymize entities and ensures the same input always gets the same anonymized output by storing hashes in a shared mapping.

In [17]:
from presidio_anonymizer.operators import Operator, OperatorType
import hashlib
from typing import Dict

class ReAnonymizer(Operator):
    """
    Anonymizer that replaces text with a reusable SHA-256 hash,
    stored in a shared mapping dict.
    """

    def operate(self, text: str, params: Dict = None) -> str:
        entity_type = params.get("entity_type", "DEFAULT")
        mapping = params.get("entity_mapping")

        if mapping is None:
            raise ValueError("Missing `entity_mapping` in params")

        # Check if already hashed
        if entity_type in mapping and text in mapping[entity_type]:
            return mapping[entity_type][text]

        # Hash and store
        hashed = "<HASH_" + hashlib.sha256(text.encode()).hexdigest()[:10] + ">"
        mapping.setdefault(entity_type, {})[text] = hashed
        return hashed

    def validate(self, params: Dict = None) -> None:
        if "entity_mapping" not in params:
            raise ValueError("You must pass an 'entity_mapping' dictionary.")

    def operator_name(self) -> str:
        return "reanonymizer"

    def operator_type(self) -> OperatorType:
        return OperatorType.Anonymize


## 4.2 Define Custom PII Recognizers for PAN and Aadhaar Numbers
We define two custom regex-based PatternRecognizers — one for Indian PAN numbers and one for Aadhaar numbers. These will detect custom PII entities in your text.

In [19]:
from presidio_analyzer import AnalyzerEngine, PatternRecognizer, Pattern

# Define custom recognizers
pan_recognizer = PatternRecognizer(
    supported_entity="IND_PAN",
    name="PAN Recognizer",
    patterns=[Pattern(name="pan", regex=r"\b[A-Z]{5}[0-9]{4}[A-Z]\b", score=0.8)],
    supported_language="en"
)

aadhaar_recognizer = PatternRecognizer(
    supported_entity="AADHAAR",
    name="Aadhaar Recognizer",
    patterns=[Pattern(name="aadhaar", regex=r"\b\d{4}[- ]?\d{4}[- ]?\d{4}\b", score=0.8)],
    supported_language="en"
)

## 4.3 Set Up Analyzer and Anonymizer Engines
Here we set up the Presidio AnalyzerEngine, register the custom recognizers, and add the custom anonymizer to the AnonymizerEngine.

In [20]:
from presidio_anonymizer import AnonymizerEngine, OperatorConfig

# Initialize analyzer and register custom recognizers
analyzer = AnalyzerEngine()
analyzer.registry.add_recognizer(pan_recognizer)
analyzer.registry.add_recognizer(aadhaar_recognizer)

# Initialize anonymizer and add custom operator
anonymizer = AnonymizerEngine()
anonymizer.add_anonymizer(ReAnonymizer)

# Shared mapping dictionary for consistent re-anonymization
entity_mapping = {}

## 4.4 Analyze and Anonymize Input Texts
We analyze two separate texts that both include the same PAN and Aadhaar values. The custom operator ensures they’re anonymized consistently across both inputs.

In [21]:
from pprint import pprint

# Example texts
text1 = "My PAN is ABCDE1234F and Aadhaar number is 1234-5678-9123."
text2 = "His Aadhaar is 1234-5678-9123 and PAN is ABCDE1234F."

# Analyze and anonymize first text
results1 = analyzer.analyze(text=text1, language="en")
anon1 = anonymizer.anonymize(
    text1,
    results1,
    {
        "DEFAULT": OperatorConfig("reanonymizer", {"entity_mapping": entity_mapping})
    }
)

# Analyze and anonymize second text
results2 = analyzer.analyze(text=text2, language="en")
anon2 = anonymizer.anonymize(
    text2,
    results2,
    {
        "DEFAULT": OperatorConfig("reanonymizer", {"entity_mapping": entity_mapping})
    }
)

## 4.5 View Anonymization Results and Mapping
Finally, we print both anonymized outputs and inspect the mapping used internally to maintain consistent hashes across values.

In [22]:
print("📄 Original 1:", text1)
print("🔐 Anonymized 1:", anon1.text)
print("📄 Original 2:", text2)
print("🔐 Anonymized 2:", anon2.text)

print("\n📦 Mapping used:")
pprint(entity_mapping)

📄 Original 1: My PAN is ABCDE1234F and Aadhaar number is 1234-5678-9123.
🔐 Anonymized 1: My PAN is <HASH_6442fd73a9> and Aadhaar number is <HASH_08e9d6b34c>.
📄 Original 2: His Aadhaar is 1234-5678-9123 and PAN is ABCDE1234F.
🔐 Anonymized 2: His Aadhaar is <HASH_08e9d6b34c> and PAN is <HASH_6442fd73a9>.

📦 Mapping used:
{'AADHAAR': {'1234-5678-9123': '<HASH_08e9d6b34c>'},
 'DATE_TIME': {'1234-5678-9123': '<HASH_08e9d6b34c>'},
 'IN_PAN': {'ABCDE1234F': '<HASH_6442fd73a9>'}}
